# **GAME OF LIFE** 

### Description

[Game of Life](http://en.wikipedia.org/wiki/Conway's_Game_of_Life) (GoF) is a cellular automaton devised by the British mathematician John Horton Conway in 1970. The game is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One interacts with the Game of Life by creating an initial configuration and observing how it evolves, or, for advanced players, by creating patterns with particular properties.


The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, live or dead. Every cell interacts with its eight neighbours, which are the cells that are directly horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* Any live cell with fewer than two live neighbours dies, as if by needs caused by underpopulation.
* Any live cell with more than three live neighbours dies, as if by overcrowding.
* Any live cell with two or three live neighbours lives, unchanged, to the next generation.
* Any dead cell with exactly three live neighbours becomes a live cell.

The initial pattern constitutes the 'seed' of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed – births and deaths happen simultaneously, and the discrete moment at which this happens is sometimes called a tick. (In other words, each generation is a pure function of the one before.) The rules continue to be applied repeatedly to create further generations.

## **Module _gameoflife_** 

This module contains the core functions needed to generate the initial distribution of the cells, i.e. the initial pattern, and their future evolution controlled by the _Game of life_ rules. \
The module is divided into two submodules, which are _patterns_ and _evolution_.

## Submodule _evolution_

The module _evolution_ consists in two functions, namely _newgen_ and _evolution_. They both deal with 2D numpy arrays, which contains boolean variables: `True` stands for alive cells and `False` stands for dead cells. These numpy arrays represent the 2D grid of the _Game of life_ world in which cells live. 

### Function _newgen_

The function _newgen_ takes in input the current generation of cells and gives in output the next one, calculated through the rules of _Game of life_. Its structure is very simple. 


First, the matrix representing the current generation of cells gets padded in order to identify the borders of the corresponding grid, making a toroidal structure of the world in which they live. Then, since alive and dead cells have to respect different rules, the function creates two different arrays for their indeces: `alive_idx` and `dead_idx`. However, these arrays are traslated with respect to the original grid due to the padding procedure; hence, the function fix this by adding a traslation by [1, 1]. 


Ultimately, there is the actual next generation creation. The old generation matrix gets copied into a new one, which is then modified by the two for cicles that apply the _Game of life_ rules to each cell. The first for cicle make alive cells evolve, whereas the second make the dead cells evolve. 

In [1]:
import numpy as np
import numpy.typing as npt #For writing the data types in the function definition

def newgen(cells: npt.NDArray[np.bool_]):
    #This create a wrapped surface, where top is identified with bottom and left with right
    padded = np.pad(cells, pad_width=1, mode='wrap')
    
    #Extracts the indexes of living cells (True) and dead cells (False)
    alive_idx = np.argwhere(cells == True) 
    dead_idx = np.argwhere(cells == False) 

    #I add [1, 1] in order to traslate the indexes and make them compatible with the padded matrix
    alive_idx += np.array([1, 1])
    dead_idx += np.array([1, 1])

    #Make a copy of the grid, i.e. the next generation (necessary to make the code work)
    newgen = np.copy(cells)

    #Calculates the evolution of the living cells
    for index in alive_idx:
        neig = padded[index[0]-1:index[0]+2, index[1]-1:index[1]+2]
        #The -1 is to delete the cell I am considering from the counts of alive neighbors
        nalive = len(neig[neig == True]) - 1
        if (nalive != 2 and nalive != 3):
            newgen[index[0]-1, index[1]-1] = False

    #Calculates the evolution of the dead cells
    for index in dead_idx:
        neig = padded[index[0]-1:index[0]+2, index[1]-1:index[1]+2]
        #The -1 is to delete the cell I am considering from the counts of alive neighbors
        nalive = len(neig[neig == True]) 
        if (nalive == 3):
            newgen[index[0]-1, index[1]-1] = True

    return newgen

### Function _evolution_

This function takes in input the first generation, i.e. the original pattern, and gives in output the evolution timeline of that original pattern. The number of steps in the timeline is set by the other parameter of the _evolution_ function, which is `timesteps`.  

In [2]:
def evolution(genzero: npt.NDArray[np.bool_], timesteps: int):
    
    # Creates a list containing the configurations for each timestep in the evolution
    timeline = []
    
    # Initialize the current state with the generation zero
    current_state = genzero
    timeline.append(current_state) # Include the starting state in the timeline
    
    # Calculates the configuration for each timestep
    for t in range(timesteps):
        # ERROR FIX: We must use 'current_state' as input, not 'genzero' repeatedly
        new = newgen(cells=current_state)
        
        # Update the current state for the next iteration
        current_state = new
        
        timeline.append(new)
    
    return timeline


## Submodule _patterns_
The module _patterns_ allows the user to insert specific patterns into the grid. It relies on a dictionary called `SEED_DATA` which stores the matrix representation of various patterns, categorized into: `Still Life`, `Oscillator`, `Spaceship`, and `Complex`.

### Function _insert_pattern_
This function places a pattern into the grid at specified coordinates. It handles:
- **Toroidal wrapping**: If a pattern goes off the edge, it wraps around to the other side.
- **Rotation**: The pattern can be rotated 90, 180, or 270 degrees anticlockwise.
- **Flipping**: The pattern can be horizontally flipped.

#### Syntax
```python
insert_pattern(grid, category, name, row_origin, col_origin, rotate=0, flip=False)
```

#### Parameters
- `grid`: NumPy array representing the Game of Life world
- `category`: Pattern category (e.g., "Still Life", "Oscillator")
- `name`: Pattern name (e.g., "Glider", "Pulsar")
- `row_origin`, `col_origin`: Top-left coordinates where the pattern will be placed
- `rotate`: Number of 90° anticlockwise rotations (0-3)
- `flip`: Boolean, if True flips the pattern horizontally

#### Example Usage
```python
import numpy as np
import patterns

# Create a 50x50 grid
grid = np.zeros((50, 50), dtype=int)

# Insert a Glider at position (10, 10)
grid = patterns.insert_pattern(grid, "Spaceship", "Glider", 10, 10)

# Insert a rotated and flipped Pulsar at position (20, 20)
grid = patterns.insert_pattern(grid, "Oscillator", "Pulsar", 20, 20, rotate=1, flip=True)

# Insert a Glider Gun
grid = patterns.insert_pattern(grid, "Complex", "Glider Gun", 5, 5)
```





### Available Patterns
**Notation**: In the ASCII diagrams below, `O` represents a live cell and `.` represents a dead cell.
#### Still Lifes
Stable patterns that remain unchanged over time.

##### Block
```
OO
OO
```
**Size**: 2×2 | **Live cells**: 4

The simplest still life pattern. A perfect 2×2 square where each cell has exactly 3 living neighbors, ensuring stability according to the survival rule.

##### Beehive
```
.OO.
O..O
.OO.
```
**Size**: 3×4 | **Live cells**: 6

A hexagonal shape resembling a beehive. Each living cell has 2-3 neighbors, maintaining perfect stability.

##### Loaf
```
.OO.
O..O
.O.O
..O.
```
**Size**: 4×4 | **Live cells**: 7

Similar to the Beehive but with an asymmetric "tail" that gives it a loaf-like shape. Rows 1-2 resemble a Beehive, while rows 3-4 form the distinctive tail.

---

#### Oscillators
Patterns that return to their initial state after a fixed period.

##### Blinker (Period 2)
```
Horizontal:  OOO    →    Vertical:  .O.
                                    .O.
                                    .O.
```
**Size**: 1×3 (or 3×1) | **Live cells**: 3 | **Period**: 2

The simplest oscillator. Alternates between horizontal and vertical orientations. The center cell survives (2 neighbors), edge cells die (1 neighbor), and new cells are born above and below.

##### Toad (Period 2)
```
..O.
O..O
O..O
.O..
```
**Size**: 4×4 | **Live cells**: 6 | **Period**: 2

This pattern represents the expanded phase of the Toad oscillator, appearing as a hollow diamond-like shape. Every generation, it alternates with its compact phase, which consists of two horizontal bars of three cells offset diagonally. This rhythmic expansion and contraction mimics the movement of a breathing toad.
##### Pulsar (Period 3)
```
...OOO...OOO...
.O....O.O....O.
.O....O.O....O.
.O....O.O....O.
...OOO...OOO...
...............
...OOO...OOO...
.O....O.O....O.
.O....O.O....O.
.O....O.O....O.
...OOO...OOO...
```
**Size**: 13×13 | **Live cells**: 48 | **Period**: 3

A highly symmetric structure with 4 identical "arms" arranged in a cross formation. Each arm pulses through 3 generations, making it one of the most recognizable large oscillators.

##### Pentadecathlon (Period 15)
```
..O....O..
OO.OOOO.OO
..O....O..
```
**Size**: 3×10 | **Live cells**: 12 | **Period**: 15
The Pentadecathlon is a rare and powerful period-15 oscillator, here represented in its most compact 3x10 phase. Its structure consists of a horizontal core of twelve cells: a central row featuring two outer pairs and a four-cell inner bridge, flanked by two "wings" in the rows above and below. Throughout its 15-generation cycle, the pattern undergoes a dramatic evolution, expanding vertically to a height of 9 cells before perfectly contracting back to this initial configuration. Due to its high period and unique geometry, it is frequently used in complex Game of Life constructions as a "reflector" to change the path of traveling gliders.


#### Spaceships
Patterns that translate (move) across the grid over time.

##### Glider
```
.O.
..O
OOO
```
**Size**: 3×3 | **Live cells**: 5 | **Speed**: 1 cell diagonally every 4 generations (c/4) | **Direction**: Down-right (in this orientation)

The Glider is the smallest spaceship. It operates on a period of 4, transitioning through two distinct asymmetrical phases before returning to its original 3x3 diagonal orientation. During each full cycle, the entire structure displaces itself by one cell diagonally. Due to its simplicity and stability, it is the primary means of transmitting information across the grid in complex cellular automata constructions.

##### LWSS (Lightweight Spaceship)
```
.OOOO
O...O
....O
O..O.
```
**Size**: 4×5 | **Live cells**: 9 | **Speed**: 2 cells horizontally every 4 generations (c/2) | **Direction**: Right (in this orientation)

The LWSS is the smallest orthogonally-traveling spaceship, featuring an aerodynamic "head" in the first row followed by a body and tail. It operates on a period of 4, cycling through three intermediate asymmetrical states before returning to its original configuration. During this full cycle, the entire structure shifts 2 cells horizontally across the grid, maintaining a constant speed of c/2. As a foundational pattern, it belongs to the same family as the Medium (MWSS) and Heavy (HWSS) spaceships.

---
#### Complex Patterns




##### **Glider Gun (Gosper)**

```
........................OO..........  <-- Right Stabilizer (Eater/Block)
........................OO..........  <-- Right Stabilizer (Eater/Block)
............OO......................  <-- Queen Bee Shuttle B (Top)
...........O..O.....................  <-- Queen Bee Shuttle B (Top)
OO........O....O..OO................  <-- Left Block + Colliding Shuttles
OO........O...O.O.OO................  <-- Left Block + Colliding Shuttles
..........O....O....................  <-- Queen Bee Shuttle A (Bottom)
...........O..O.....................  <-- Queen Bee Shuttle A (Bottom)
............OO......................  <-- Queen Bee Shuttle A (Bottom)

```

**Size**: 9×36 | **Initial live cells**: 36 | **Period**: 30 | **Output**: 1 Glider every 30 generations

**Description**:
The Gosper Glider Gun was the first discovered pattern capable of infinite growth. It functions as a complex "factory" where two unstable oscillators, known as **Queen Bee shuttles**, collide rhythmically in the center.

**Dynamics and Significance**:

* **Mechanism**: Every 30 generations, the collision between the two shuttles "pinches off" a small cluster of cells that stabilizes into a **Glider**.
* **Stabilization**: The stable **Block patterns** at the edges act as "eaters" or reflectors, cleaning up debris and bouncing the shuttles back to reset the cycle.
* **Turing-Completeness**:By emitting a continuous stream of Gliders, this pattern proves the Game of Life is Turing-complete. Gliders act as data bits (1s and 0s) while their collisions simulate logical gates (AND, OR, NOT), theoretically allowing the system to perform any universal computation. As the gun cycles indefinitely, the grid's population grows linearly, providing the infinite "memory" and signal flow required for a universal computer.
* **Infinite Growth**: Because it emits a new spaceship indefinitely, the total population of the grid increases linearly over time ( Gliders after  generations).

---


### Helper Functions

#### get_available_categories()
Returns a list of all available pattern categories.

```python
categories = patterns.get_available_categories()
print(categories)
# Output: ['Still Life', 'Oscillator', 'Spaceship', 'Complex']
```

#### get_patterns_by_category(category)
Returns the names of all patterns in a specific category.

```python
oscillators = patterns.get_patterns_by_category("Oscillator")
print(oscillators)
# Output: ['Blinker', 'Toad', 'Pulsar', 'Pentadecathlon']

spaceships = patterns.get_patterns_by_category("Spaceship")
print(spaceships)
# Output: ['Glider', 'LWSS']
```

### Complete Example
```python
import numpy as np
import patterns

# Initialize grid
grid = np.zeros((100, 100), dtype=int)

# Discover available patterns
print("Categories:", patterns.get_available_categories())
print("Oscillators:", patterns.get_patterns_by_category("Oscillator"))

# Add various patterns
grid = patterns.insert_pattern(grid, "Still Life", "Block", 10, 10)
grid = patterns.insert_pattern(grid, "Oscillator", "Blinker", 20, 20)
grid = patterns.insert_pattern(grid, "Spaceship", "Glider", 30, 30)
grid = patterns.insert_pattern(grid, "Complex", "Glider Gun", 50, 10)

# Add rotated pattern
grid = patterns.insert_pattern(grid, "Spaceship", "LWSS", 40, 40, rotate=1)
```




## Submodule *Visualization*
This module deals with the graphical visualization of the numerical grid we talked about above. It is implemented in the application code (*visualization_app.py*), which uses the _gameoflife_ module to graphically simulate the grid and the cells evolution. 

### Function *create_evolution*

This function serves as the visualization engine for a **Game of Life** simulation. It transforms a raw 2D NumPy array into a dynamic `matplotlib` animation, handling both aesthetic formatting and frame-by-frame logic. \
* It takes in input the following parameters:
    * `grid`: the cells grid, either generated by *patterns* or *evolution* submodules described above.
    * `frames`: number of generations to animate.
    * `interval`: the time interval in milliseconds between frames.
* It defines the cells color, which are black for the dead cells and white for the alive cells.
* It calculates the figure dimentions starting from the grid dimentions. It also deal with possible visualization issues such as too big figures.
* It draws the grid lines and removes the grid ticks for a more clear and uniform visualization.
* It adds a title to the figure, which shows the name of the simulation, the grid dimentions and the current generation number that is being showed. 
* The *Update* function calculates the next generation using the *evolution* submodule we described above.
* In the end, it creates the animation object, containing all the objects and features previously calculated.
* It returns the animation object: `anim`

*Note*: The *is_notebook* function is only an internal check needed to distinguish between two files we used during the application development. It has no importance in the final result.

In [ ]:
COLORS = {
    "Alive" : "white"  ,        # Alive cells
    "Dead"  : "black"  ,        # Dead cells
    "Grid"  : "#505050",        # Grid color (light gray)
}

def is_notebook() -> bool:
    """
    Detects if the code is running in a Jupyter notebook environment.
    Used to determine whether to use HTML display for Jupyter notebooks or plt.show() for regular Python scripts.
    Returns:
        bool: True if running in Jupyter, False otherwise.
    """
    try:
        from IPython import get_ipython
        if get_ipython() is None:
            return False
        # Check if we're in an IPython environment (notebook or IPython terminal)
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':          # Jupyter notebook
            return True
        elif shell == 'TerminalInteractiveShell':   # IPython terminal
            return False
        else:
            return False
    except (ImportError, NameError):
        return False

def create_evolution(grid: np.ndarray, frames: int, interval: int) -> animation.FuncAnimation:
    """
    Creates the Game of Life grid evolution using matplotlib.
    Helper function to be used by plot_evolution.
    Args (passed by plot_evolution):
        grid (np.ndarray): 2D array (either boolean or numeric) representing the initial state.
        frames (int): Number of generations to animate.
        interval (int): Time in milliseconds between frames.
    Returns:
        animation.FuncAnimation: The animation object that can be displayed in Jupyter.
    """

    # 1. Create a personalized colormap
    colors_list = [COLORS["Dead"], COLORS["Alive"]]
    cmap = mcolors.ListedColormap(colors_list)
    


    # 2. Calculate figure size based on grid dimensions for better visualization
    # First, get number of rows and columns and we define how big each cell should be
    rows, cols = grid.shape
    cell_size_inches = 0.5
    
    # Then we calculate width and height of the figure adding a small margin for 
    # title and axes
    w = cols * cell_size_inches + 1.5
    h = rows * cell_size_inches + 1.5
    
    # Finally, we limit the maximum size to avoid too large figures
    max_size = 7
    w = min(w, max_size)
    h = min(h, max_size)



    # 3. Visualization 
    #    We use imshow because it's very efficient for displaying 2D arrays.
    #    We add interpolation='nearest' to avoid blurring of the cells.
    #    We add .astype(int) in case the grid is boolean, so it coverts False->0, True->1
    fig, ax = plt.subplots(figsize=(w,h), dpi=120)    
    img = ax.imshow(grid.astype(int), cmap=cmap, interpolation='nearest')



    # 4. Aesthetic adjustments
    #    First, we use major tick positions to definire where grid lines should be drawn
    #    -0.5 is used to center the grid lines between the cells
    ax.set_xticks(np.arange(-0.5, cols, 1))
    ax.set_yticks(np.arange(-0.5, rows, 1))
    ax.grid(which='major', color=COLORS["Grid"], linestyle='-', linewidth=1)
     
    # Then, we remove the ticks marks and labels for a cleaner look (grid lines remain visible).
    # We also set the title, pad is used to add some space between title and grid
    ax.tick_params(which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    title = ax.set_title(f"Game of Life - Dimension: {rows}x{cols}\nGeneration: 1", 
                         fontsize=15, color='black', pad=20)


    # 5. Creating the animation
    #    First, we create a function to "update" every frame
    def update(frame):
        """
        This function is called by FuncAnimation for every frame.
        """
        # 1. Compute the next generation of the grid. 
        #    Keyword nonlocal "tells" Python to edit the variable "grid" from
        #    the external scope, avoiding creating a new local variable
        nonlocal grid
        grid = evo.newgen(grid)

        # 2. Update the image data and title text. 
        #    Instead of plotting again, we just update the data of the existing image
        img.set_data(grid.astype(int))
        title.set_text(f"Game of Life - Dimension: {rows}x{cols}\nGeneration: {frame+1}")

        # Return a list of elements that have changed to be used by blit for optimization.
        return img, title

    plt.tight_layout()

    # Then, we create the animation object
    # Use blit=True only in notebooks (where to_jshtml() is used) for better performance
    # In regular Python scripts, blit=True causes the grid to disappear after the first frame
    # because the grid is not returned by the update function
    use_blit = is_notebook()
    anim = animation.FuncAnimation(fig, update, frames=frames, interval=interval, 
                                   blit=use_blit, repeat=False)
    
    # Close the figure only in notebook environments to prevent static image display
    # In regular Python scripts, we need to keep it open for plt.show()
    if is_notebook():
        plt.close(fig)
    
    return anim

### Function *plot_evolution*
This functions plot the graphical animation we have just created with the *create_evolution* function above.
* It takes in input the following parameters:
    * `grid`: the cells grid, either generated by *patterns* or *evolution* submodules described above.
    * `frames`: number of generations to animate.
    * `FPS`: frames per second for the animation. It is then converted in `interval` for consistency with the *create_evolution* function
* It creates the animation using the *create_evolution* function. 
* As we wrote in the note above, it does a check which was important during the development phase.
* It shows the plot using `matplotlib.pyplot`.

In [ ]:
def plot_evolution(grid: np.ndarray, frames: int, FPS: int) -> None:
    """
    Plots the Game of Life grid evolution.
    Automatically detects the environment and uses:
    - HTML display for Jupyter notebooks
    - plt.show() for regular Python scripts
    
    Args:
        grid (np.ndarray): 2D array representing the initial grid state.
        frames (int): Number of generations to animate.
        FPS (int): Frames per second for the animation (converted in "interval": delay
                   between frames in milliseconds).
    """

    # 1. Use the helper function to create the animation object
    interval = 1000 // FPS  # Convert FPS to interval in milliseconds
    anim = create_evolution(grid, frames, interval=interval)

    # 2. Display the animation based on the environment
    if is_notebook():
        # In Jupyter notebook: use HTML display with styling
        try:
            from IPython.display import HTML, display
            styled_html = f"""
                <div style="max-width: 600px; width: 100%; margin: 0 auto;">
                    {anim.to_jshtml()}
                </div>
            """
            display(HTML(styled_html))
        except ImportError:
            # Fallback if IPython is not available (shouldn't happen if is_notebook() returned True)
            print("Warning: IPython not available, using plt.show() instead")
            plt.show()
    else:
        # In regular Python script: use matplotlib's native display
        plt.show()

    return None